# SBET - Smoothed Best Estimate of Trajectory
Sowohl mit geopandas als auch mit pyproj.Transformer wird die Höhe nicht umgerechnet???

In [1]:
import pdal 
import numpy as np
import os
import pandas as pd
import geopandas as gpd
from pyproj import Transformer 

In [2]:
filename = 'sbet_merge_merged_2.out'
folder = '/media/riannek/minimax/gleis/gps/run24'

pipeline = pdal.Pipeline([pdal.Reader(os.path.join(folder, filename), tag="input1", type="readers.sbet")])
pipeline.execute()
points = pipeline.arrays[0]
points.dtype # Column names and types

dtype([('GpsTime', '<f8'), ('Y', '<f8'), ('X', '<f8'), ('Z', '<f8'), ('XVelocity', '<f8'), ('YVelocity', '<f8'), ('ZVelocity', '<f8'), ('Roll', '<f4'), ('Pitch', '<f4'), ('Azimuth', '<f8'), ('WanderAngle', '<f8'), ('XBodyAccel', '<f8'), ('YBodyAccel', '<f8'), ('ZBodyAccel', '<f8'), ('XBodyAngRate', '<f8'), ('YBodyAngRate', '<f8'), ('ZBodyAngRate', '<f8')])

In [3]:
df = pd.DataFrame(points)
df.head()

,GpsTime,Y,X,Z,XVelocity,YVelocity,ZVelocity,Roll,Pitch,Azimuth,WanderAngle,XBodyAccel,YBodyAccel,ZBodyAccel,XBodyAngRate,YBodyAngRate,ZBodyAngRate
0,199390.055789,48.195315,11.505046,550.499792,-0.015664,-0.100388,0.004476,-0.531755,-0.807040,309.406218,0.0,-0.103675,0.057052,-0.017389,-0.230186,-0.118725,0.216647
1,199390.060798,48.195315,11.505046,550.499722,-0.015617,-0.099892,0.004619,-0.532249,-0.807146,309.406615,0.0,-0.024763,-0.065341,-0.067080,-0.097806,-0.021781,0.079136
2,199390.065798,48.195315,11.505046,550.499659,-0.015199,-0.099427,0.004673,-0.531709,-0.806434,309.405933,0.0,0.009842,-0.025161,-0.075227,0.105897,0.143775,-0.134911
3,199390.070799,48.195315,11.505046,550.499702,-0.014846,-0.099716,0.004733,-0.531104,-0.805610,309.405393,0.0,-0.100881,0.131971,-0.020714,0.119401,0.165830,-0.106522
4,199390.075799,48.195315,11.505046,550.499745,-0.014841,-0.100557,0.005076,-0.531129,-0.805565,309.405562,0.0,-0.207546,0.174118,-0.005978,-0.004612,0.008719,0.033830


In [4]:
points.shape 

(5572516,)

In [5]:
transformer = Transformer.from_crs("EPSG:4326", "EPSG:5684", always_xy=True)

In [6]:
xyz = np.array([df['X'], df['Y'], df['Z']]).T

for i in range(xyz.shape[0]):
    xyz[i] = transformer.transform(*xyz[i])

In [7]:
geometry = gpd.points_from_xy(x=xyz[:,0], y=xyz[:,1], z=xyz[:,2], crs='EPSG:5684')
gdf = gpd.GeoDataFrame(df, geometry=geometry)
gdf.head()

,GpsTime,Y,X,Z,XVelocity,YVelocity,ZVelocity,Roll,Pitch,Azimuth,WanderAngle,XBodyAccel,YBodyAccel,ZBodyAccel,XBodyAngRate,YBodyAngRate,ZBodyAngRate,geometry
0,199390.055789,48.195315,11.505046,550.499792,-0.015664,-0.100388,0.004476,-0.531755,-0.807040,309.406218,0.0,-0.103675,0.057052,-0.017389,-0.230186,-0.118725,0.216647,POINT Z (4463311.002 5339821.304 550.5)
1,199390.060798,48.195315,11.505046,550.499722,-0.015617,-0.099892,0.004619,-0.532249,-0.807146,309.406615,0.0,-0.024763,-0.065341,-0.067080,-0.097806,-0.021781,0.079136,POINT Z (4463311.005 5339821.304 550.5)
2,199390.065798,48.195315,11.505046,550.499659,-0.015199,-0.099427,0.004673,-0.531709,-0.806434,309.405933,0.0,0.009842,-0.025161,-0.075227,0.105897,0.143775,-0.134911,POINT Z (4463311.008 5339821.304 550.5)
3,199390.070799,48.195315,11.505046,550.499702,-0.014846,-0.099716,0.004733,-0.531104,-0.805610,309.405393,0.0,-0.100881,0.131971,-0.020714,0.119401,0.165830,-0.106522,POINT Z (4463311.008 5339821.304 550.5)
4,199390.075799,48.195315,11.505046,550.499745,-0.014841,-0.100557,0.005076,-0.531129,-0.805565,309.405562,0.0,-0.207546,0.174118,-0.005978,-0.004612,0.008719,0.033830,POINT Z (4463311.009 5339821.304 550.5)


In [ ]:
gdf.to_file("test.gpkg", driver="GPKG", layer='sbet')

In [10]:
pt = points[0]
print(pt['X'], pt['Y'], pt['Z'])

11.505046168093596 48.1953151694577 550.4997917973492


In [11]:
transformer.transform(pt['X'], pt['Y'], pt['Z'])

(4463311.001699163, 5339821.304146841, 550.4997917973492)

In [12]:
transformer2 = Transformer.from_crs("EPSG:4326", "EPSG:9934", always_xy=True)
transformer2.transform(pt['X'], pt['Y'], pt['Z'])

(4463311.001699163, 5339821.304146841, 550.4997917973492)